## <div align="center"> PRESENTASI LAPORAN AKHIR MACHINE LEARNING </div>
## <div align="center"> Judul: Klasifikasi Saham IDX kepada Listing Board berdasarkan 5 Faktor Utama </div>
### <div align="center"> oleh Kelompok 5 </div>
### <div align="center"> Semester Genap 2022/2023 </div>

## Melakukan import library yang digunakan

In [1]:
import pandas as pd
import numpy as np
import re
from operator import eq
import os.path

# EDA

## Menampilkan data

In [2]:
dftarSaham = pd.read_csv('/Users/ASUS/Documents/Anaconda/Project ML/DaftarSaham.csv')
dftarSaham.head()

,Code,Name,ListingDate,Shares,ListingBoard,Sector,LastPrice,MarketCap,MinutesFirstAdded,MinutesLastUpdated,HourlyFirstAdded,HourlyLastUpdated,DailyFirstAdded,DailyLastUpdated
0,AALI,Astra Agro Lestari Tbk.,1997-12-09,1.924688e+09,Utama,Consumer Non-Cyclicals,8000.0,1.539751e+13,2021-11-01 09:00:00,2023-01-06 15:59:00,2020-04-16 09:00:00,2023-01-06 15:00:00,2001-04-16,2023-01-06
1,ABBA,Mahaka Media Tbk.,2002-04-03,3.935893e+09,Pengembangan,Consumer Cyclicals,142.0,5.588968e+11,2021-11-01 09:00:00,2023-01-06 15:59:00,2020-04-16 09:00:00,2023-01-06 15:00:00,2002-04-03,2023-01-06
2,ABDA,Asuransi Bina Dana Arta Tbk.,1989-07-06,6.208067e+08,Pengembangan,Financials,6700.0,4.159405e+12,2021-11-01 09:00:00,2022-11-11 15:59:00,2020-04-16 09:00:00,2022-11-11 16:00:00,2001-04-16,2023-01-06
3,ABMM,ABM Investama Tbk.,2011-12-06,2.753165e+09,Utama,Industrials,3050.0,8.397153e+12,2021-11-01 09:00:00,2023-01-06 15:59:00,2020-04-16 09:00:00,2023-01-06 15:00:00,2011-12-06,2023-01-06
4,ACES,Ace Hardware Indonesia Tbk.,2007-11-06,1.715000e+10,Utama,Consumer Cyclicals,490.0,8.403500e+12,2021-11-01 09:00:00,2023-01-06 15:59:00,2020-04-16 09:00:00,2023-01-06 15:00:00,2007-11-06,2023-01-06


## Menampilkan informasi kolom

In [3]:
dftarSaham.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 829 entries, 0 to 828
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Code                829 non-null    object 
 1   Name                829 non-null    object 
 2   ListingDate         829 non-null    object 
 3   Shares              829 non-null    float64
 4   ListingBoard        829 non-null    object 
 5   Sector              829 non-null    object 
 6   LastPrice           824 non-null    float64
 7   MarketCap           824 non-null    float64
 8   MinutesFirstAdded   787 non-null    object 
 9   MinutesLastUpdated  787 non-null    object 
 10  HourlyFirstAdded    806 non-null    object 
 11  HourlyLastUpdated   806 non-null    object 
 12  DailyFirstAdded     824 non-null    object 
 13  DailyLastUpdated    824 non-null    object 
dtypes: float64(3), object(11)
memory usage: 90.8+ KB


In [4]:
nameExist = dftarSaham['Code'].values
nameExist[:15]

array(['AALI', 'ABBA', 'ABDA', 'ABMM', 'ACES', 'ACST', 'ADCP', 'ADES',
       'ADHI', 'ADMF', 'ADMG', 'ADMR', 'ADRO', 'AGAR', 'AGII'],
      dtype=object)

## Melakukan Feature engineering

In [5]:
idx = 0

for i in nameExist:
    
    name = i
    path = '/Users/ASUS/Documents/Anaconda/Project ML/DetailSaham/' + name + '.csv'
    
    if os.path.isfile(path) == True:
        temp = pd.read_csv(path)
        pathSave = '/Users/ASUS/Documents/Anaconda/Project ML/SahamConfirmed/' + name + '.csv'
        temp.to_csv(pathSave, index = False)
        idx += 1

    elif os.path.isfile(path) == False:
        dftarSaham.drop(index = dftarSaham.iloc[idx, :].name, inplace = True)
        idx += 1
        idx -= 1
    
dftarSaham.reset_index()

,index,Code,Name,ListingDate,Shares,ListingBoard,Sector,LastPrice,MarketCap,MinutesFirstAdded,MinutesLastUpdated,HourlyFirstAdded,HourlyLastUpdated,DailyFirstAdded,DailyLastUpdated
0,0,AALI,Astra Agro Lestari Tbk.,1997-12-09,1.924688e+09,Utama,Consumer Non-Cyclicals,8000.0,1.539751e+13,2021-11-01 09:00:00,2023-01-06 15:59:00,2020-04-16 09:00:00,2023-01-06 15:00:00,2001-04-16,2023-01-06
1,1,ABBA,Mahaka Media Tbk.,2002-04-03,3.935893e+09,Pengembangan,Consumer Cyclicals,142.0,5.588968e+11,2021-11-01 09:00:00,2023-01-06 15:59:00,2020-04-16 09:00:00,2023-01-06 15:00:00,2002-04-03,2023-01-06
2,2,ABDA,Asuransi Bina Dana Arta Tbk.,1989-07-06,6.208067e+08,Pengembangan,Financials,6700.0,4.159405e+12,2021-11-01 09:00:00,2022-11-11 15:59:00,2020-04-16 09:00:00,2022-11-11 16:00:00,2001-04-16,2023-01-06
3,3,ABMM,ABM Investama Tbk.,2011-12-06,2.753165e+09,Utama,Industrials,3050.0,8.397153e+12,2021-11-01 09:00:00,2023-01-06 15:59:00,2020-04-16 09:00:00,2023-01-06 15:00:00,2011-12-06,2023-01-06
4,4,ACES,Ace Hardware Indonesia Tbk.,2007-11-06,1.715000e+10,Utama,Consumer Cyclicals,490.0,8.403500e+12,2021-11-01 09:00:00,2023-01-06 15:59:00,2020-04-16 09:00:00,2023-01-06 15:00:00,2007-11-06,2023-01-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
733,823,YULE,Yulie Sekuritas Indonesia Tbk.,2004-12-10,1.785000e+09,Pengembangan,Financials,2230.0,3.980550e+12,2021-11-01 09:00:00,2023-01-06 15:59:00,2020-04-16 09:00:00,2023-01-06 15:00:00,2004-12-10,2023-01-06
734,825,ZBRA,Dosni Roha Indonesia Tbk.,1991-08-01,2.510706e+09,Pengembangan,Industrials,540.0,1.355781e+12,2021-11-01 09:00:00,2023-01-06 15:59:00,2020-04-16 09:00:00,2023-01-06 15:00:00,2001-04-16,2023-01-06
735,826,ZINC,Kapuas Prima Coal Tbk.,2017-10-16,2.525000e+10,Pengembangan,Basic Materials,50.0,1.262500e+12,2021-11-01 09:00:00,2023-01-06 15:59:00,2020-04-16 09:00:00,2023-01-06 15:00:00,2017-10-16,2023-01-06
736,827,ZONE,Mega Perintis Tbk.,2018-12-12,8.701715e+08,Pengembangan,Consumer Cyclicals,1205.0,1.048557e+12,2021-11-01 09:00:00,2023-01-06 15:59:00,2020-04-16 09:00:00,2023-01-06 15:00:00,2018-12-12,2023-01-06


In [6]:
board = dftarSaham[['Code', 'Name', 'ListingDate', 'Shares', 'ListingBoard', 'MarketCap', 'DailyLastUpdated']]
board['Operational Lifetime'] = np.nan
board['Avg. Value'] = np.nan
board['Avg. Frequency'] = np.nan
board.head()

/var/folders/v5/ffmz1k395r39ysqjvdbwnnp40000gn/T/ipykernel_51540/3069793789.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  board['Operational Lifetime'] = np.nan
/var/folders/v5/ffmz1k395r39ysqjvdbwnnp40000gn/T/ipykernel_51540/3069793789.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  board['Avg. Value'] = np.nan
/var/folders/v5/ffmz1k395r39ysqjvdbwnnp40000gn/T/ipykernel_51540/3069793789.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc

,Code,Name,ListingDate,Shares,ListingBoard,MarketCap,DailyLastUpdated,Operational Lifetime,Avg. Value,Avg. Frequency
0,AALI,Astra Agro Lestari Tbk.,1997-12-09,1.924688e+09,Utama,1.539751e+13,2023-01-06,NaN,NaN,NaN
1,ABBA,Mahaka Media Tbk.,2002-04-03,3.935893e+09,Pengembangan,5.588968e+11,2023-01-06,NaN,NaN,NaN
2,ABDA,Asuransi Bina Dana Arta Tbk.,1989-07-06,6.208067e+08,Pengembangan,4.159405e+12,2023-01-06,NaN,NaN,NaN
3,ABMM,ABM Investama Tbk.,2011-12-06,2.753165e+09,Utama,8.397153e+12,2023-01-06,NaN,NaN,NaN
4,ACES,Ace Hardware Indonesia Tbk.,2007-11-06,1.715000e+10,Utama,8.403500e+12,2023-01-06,NaN,NaN,NaN


In [7]:
j = 0

columnsv2 = board['Code'].values

for i in columnsv2:
    
    name = i
    path = '/Users/ASUS/Documents/Anaconda/Project ML/SahamConfirmed/' + name + '.csv' 
    temp = pd.read_csv(path)

    board.iloc[j, 8]  = temp['value'].mean()
    board.iloc[j, 9]  = temp['frequency'].mean()
    
    j += 1
    
board.head()

,Code,Name,ListingDate,Shares,ListingBoard,MarketCap,DailyLastUpdated,Operational Lifetime,Avg. Value,Avg. Frequency
0,AALI,Astra Agro Lestari Tbk.,1997-12-09,1.924688e+09,Utama,1.539751e+13,2023-01-06,NaN,2.075530e+10,1690.068894
1,ABBA,Mahaka Media Tbk.,2002-04-03,3.935893e+09,Pengembangan,5.588968e+11,2023-01-06,NaN,7.467397e+09,2493.162839
2,ABDA,Asuransi Bina Dana Arta Tbk.,1989-07-06,6.208067e+08,Pengembangan,4.159405e+12,2023-01-06,NaN,2.695621e+06,0.578288
3,ABMM,ABM Investama Tbk.,2011-12-06,2.753165e+09,Utama,8.397153e+12,2023-01-06,NaN,6.810038e+07,32.565762
4,ACES,Ace Hardware Indonesia Tbk.,2007-11-06,1.715000e+10,Utama,8.403500e+12,2023-01-06,NaN,3.303881e+10,4613.549061


In [8]:
board.dropna(subset=['DailyLastUpdated'], inplace = True)
board.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 733 entries, 0 to 828
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Code                  733 non-null    object 
 1   Name                  733 non-null    object 
 2   ListingDate           733 non-null    object 
 3   Shares                733 non-null    float64
 4   ListingBoard          733 non-null    object 
 5   MarketCap             733 non-null    float64
 6   DailyLastUpdated      733 non-null    object 
 7   Operational Lifetime  0 non-null      float64
 8   Avg. Value            733 non-null    float64
 9   Avg. Frequency        733 non-null    float64
dtypes: float64(5), object(5)
memory usage: 63.0+ KB


/var/folders/v5/ffmz1k395r39ysqjvdbwnnp40000gn/T/ipykernel_51540/953153183.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  board.dropna(subset=['DailyLastUpdated'], inplace = True)


In [9]:
board['ListingDate'] = pd.to_datetime(board['ListingDate'])
board['DailyLastUpdated'] = pd.to_datetime(board['DailyLastUpdated'])

board['Operational Lifetime'] = board['DailyLastUpdated'] - board['ListingDate']
board['Operational Lifetime'] = board['Operational Lifetime'].astype(str)
board['Operational Lifetime'] = board['Operational Lifetime'].str.replace('days$', '')
board['Operational Lifetime'] = board['Operational Lifetime'].astype(float)
board

/var/folders/v5/ffmz1k395r39ysqjvdbwnnp40000gn/T/ipykernel_51540/2301533790.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  board['ListingDate'] = pd.to_datetime(board['ListingDate'])
/var/folders/v5/ffmz1k395r39ysqjvdbwnnp40000gn/T/ipykernel_51540/2301533790.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  board['DailyLastUpdated'] = pd.to_datetime(board['DailyLastUpdated'])
/var/folders/v5/ffmz1k395r39ysqjvdbwnnp40000gn/T/ipykernel_51540/2301533790.py:4: SettingWithCopyWarning: 
A value is trying t

,Code,Name,ListingDate,Shares,ListingBoard,MarketCap,DailyLastUpdated,Operational Lifetime,Avg. Value,Avg. Frequency
0,AALI,Astra Agro Lestari Tbk.,1997-12-09,1.924688e+09,Utama,1.539751e+13,2023-01-06,9159.0,2.075530e+10,1690.068894
1,ABBA,Mahaka Media Tbk.,2002-04-03,3.935893e+09,Pengembangan,5.588968e+11,2023-01-06,7583.0,7.467397e+09,2493.162839
2,ABDA,Asuransi Bina Dana Arta Tbk.,1989-07-06,6.208067e+08,Pengembangan,4.159405e+12,2023-01-06,12237.0,2.695621e+06,0.578288
3,ABMM,ABM Investama Tbk.,2011-12-06,2.753165e+09,Utama,8.397153e+12,2023-01-06,4049.0,6.810038e+07,32.565762
4,ACES,Ace Hardware Indonesia Tbk.,2007-11-06,1.715000e+10,Utama,8.403500e+12,2023-01-06,5540.0,3.303881e+10,4613.549061
...,...,...,...,...,...,...,...,...,...,...
823,YULE,Yulie Sekuritas Indonesia Tbk.,2004-12-10,1.785000e+09,Pengembangan,3.980550e+12,2023-01-06,6601.0,1.004773e+08,6.283925
825,ZBRA,Dosni Roha Indonesia Tbk.,1991-08-01,2.510706e+09,Pengembangan,1.355781e+12,2023-01-06,11481.0,1.284895e+09,398.513570
826,ZINC,Kapuas Prima Coal Tbk.,2017-10-16,2.525000e+10,Pengembangan,1.262500e+12,2023-01-06,1908.0,4.796663e+10,4591.229645
827,ZONE,Mega Perintis Tbk.,2018-12-12,8.701715e+08,Pengembangan,1.048557e+12,2023-01-06,1486.0,6.933867e+07,36.444676


## Mengimport data kedalam CSV

In [10]:
temp = board
name = 'board'
pathSave = '/Users/ASUS/Documents/Anaconda/Project ML/' + name + '.csv'
temp.to_csv(pathSave, index = False)